## 1. Анализ исходной выборки

In [1]:
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [4]:
# в dataset/utils:
def get_index(folder_name):
    splited_name = folder_name.split(' ')
    mark, model = splited_name[0], splited_name[1]
    rakurs = 'back' if 'back' in folder_name else 'normal'
    truck = 'truck' if 'truck' in folder_name else 'no_truck'
    index = (mark, model, rakurs, truck)
    
# в dataset/utils:
def get_indexes(folders: list):
    indexes = [get_index(fold_n) for fold_n in folders]
    return indexes